In [ ]:
class BiLSTMEmbedding(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(BiLSTMEmbedding, self).__init__()
        self.bilstm = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim, batch_first=True, bidirectional=True)

    def forward(self, x):
        # BiLSTM 前向传播
        _, (h_n, _) = self.bilstm(x)  # h_n: [2, B, hidden_dim]
        # 拼接前向和后向的隐藏状态
        h_L = torch.cat((h_n[0], h_n[1]), dim=-1)  # h_L: [B, 2 * hidden_dim]
        return h_L

In [1]:
from torch_geometric.nn import SAGEConv

class GNNEmbedding(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(GNNEmbedding, self).__init__()
        # GraphSAGE 两层卷积
        self.conv1 = SAGEConv(input_dim, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, hidden_dim)

    def forward(self, x, edge_index):
        # 图嵌入传播
        h = self.conv1(x, edge_index)  # 第一层传播
        h = torch.relu(h)
        h = self.conv2(h, edge_index)  # 第二层传播
        return h  # 返回图节点嵌入

ModuleNotFoundError: No module named 'torch_geometric'

In [ ]:
class StaticEmbedding(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(StaticEmbedding, self).__init__()
        self.fc = nn.Linear(input_dim, hidden_dim)

    def forward(self, s):
        return torch.relu(self.fc(s))  # 静态嵌入

In [ ]:
class CombinedPrediction(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(CombinedPrediction, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, h_L, h_N, h_S):
        h_C = torch.cat((h_L, h_N, h_S), dim=-1)  # 拼接
        h = torch.relu(self.fc1(h_C))
        y_hat = self.fc2(h)
        return y_hat

In [ ]:
class LSTMGNNModel(nn.Module):
    def __init__(self, time_input_dim, static_input_dim, lstm_hidden_dim, gnn_hidden_dim, fc_hidden_dim, output_dim):
        super(LSTMGNNModel, self).__init__()
        self.lstm_embedding = BiLSTMEmbedding(time_input_dim, lstm_hidden_dim)
        self.gnn_embedding = GNNEmbedding(2 * lstm_hidden_dim, gnn_hidden_dim)
        self.static_embedding = StaticEmbedding(static_input_dim, lstm_hidden_dim)
        self.prediction = CombinedPrediction(2 * lstm_hidden_dim + gnn_hidden_dim + lstm_hidden_dim, fc_hidden_dim, output_dim)

    def forward(self, time_series, static_input, edge_index):
        h_L = self.lstm_embedding(time_series)  # 时间序列嵌入
        h_N = self.gnn_embedding(h_L, edge_index)  # 图嵌入
        h_S = self.static_embedding(static_input)  # 静态嵌入
        y_hat = self.prediction(h_L, h_N, h_S)  # 预测
        return y_hat

In [ ]:
criterion = nn.MSELoss()

# 损失权重
alpha = 0.5
def combined_loss(y_hat_gnn, y_hat_lstm, y_true):
    loss_lstm_gnn = criterion(y_hat_gnn, y_true)
    loss_lstm = criterion(y_hat_lstm, y_true)
    return loss_lstm_gnn + alpha * loss_lstm

In [ ]:
model = LSTMGNNModel(
    time_input_dim=34,
    static_input_dim=92,
    lstm_hidden_dim=64,
    gnn_hidden_dim=32,
    fc_hidden_dim=128,
    output_dim=1
)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)